In [1]:
import pandas as pa
import numpy as np
import pdb
import sys
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

##    Helper Funcitons (Do not edit) 

The Tree_node class represents one Node in a Decisiontree
Each node holds a left and a right child if it is not a leaf. If it is a leaf it will contain the partition of the 
original dataset corresponding to the respective leaf.
In a fully grown tree every leaf is pure with respect to the goal variable 
Each node needs to have a Split that describes how the dataset is partitioned at a specific Node. It is a python tuple 
containing:
The variable in which the dataset is split
The cutoff value for the split
The goal variable for which the split is optimized
for example ('safety', 'high', 'rating') means that the dataset will be split into a partition where safety is high
and a rest.

In [61]:
class Tree_node:
    original_Data = None
    
    def __init__(self,split=None,right_child=None, left_child=None):
        self.split=split
        self.right_child = right_child
        self.left_child = left_child

    # returns the returns the child in which obs belongs
    def return_child(self,obs):
        column = self.split[0]
                        
        if self.is_categorical(column):
            if obs[column] == self.split[1]:
                return self.right_child
            else:
                return self.left_child
            
        else:
            if obs[column] >= self.split[1]:
                return self.right_child
            else:
                return self.left_child
    
    # returns an estimate for the goal variable of obs
    def classify(self, obs):
        
        child = self.return_child(obs)
        
        if child.__class__.__name__ == 'Tree_node':
            return child.classify(obs)
    
        # split[2] is rating
        target_col = self.split[2]
        
        if self.is_categorical(target_col):
            #print("majority vote")
            return child[target_col].value_counts().keys()[0]
        else:
            #print("average")
            return np.average(child[~child[target_col].isnull()]["age"])
 

    def is_categorical(self, column):
        category=True

        if not Tree_node.original_Data[column].dtype.name == "category":
            category = False
        return category
    
# returns the giny impurity for data with respect to column (i.e. use your goal variable as column here)    
def gini_impurity(data, column, weights=None):
    try:
        # example: 
        # counts:
        # unacc    868
        # acc      318
        # good      58
        # vgood     51

        # data.shape[0]:
        # 1295
        
        # probs:
        # unacc    0.670270
        # acc      0.245560
        # good     0.044788
        # vgood    0.039382
        
        counts = uniquecounts(data, column)
        probs = counts/data.shape[0]
        
        if len(probs) == 1:
            prob_obs = np.ones(data.shape[0])
        else:
            # probs.index:
            # example:
            # CategoricalIndex(['unacc', 'acc', 'good', 'vgood'], categories=['acc', 'good', 'unacc', 'vgood'], ordered=False, dtype='category')
            
            la1 = lambda x: probs[probs.index == x][0]

            # prob_obs:
            # example:
            # [0.67027027 0.67027027 0.67027027 ... 0.24555985 0.67027027 0.24555985]

            prob_obs = np.array(list(map(la1, data[column])))
            
            prob_obs = np.square(prob_obs)

        if weights is None:
            weights = np.ones(data.shape[0])
            
        # weights
        # example: 
        # 0.6839147891222874     
        weights = weights/sum(weights)
        
        return 1-sum(weights*prob_obs)
    except:
        print("Unexpected error:", sys.exc_info()[0])
        raise
       

# Count ocurrences of goal variable
def uniquecounts(data, column):
   val_cnt = data[column].value_counts()
   return val_cnt.drop(val_cnt[val_cnt == 0].index)


# This is a helper function, that partitions a dataset with respect to a given variable and a cutoff value
def divideset(in_set, column, value):
   # Make a function that tells us if a row is in
   # the first group (true) or the second group (false)
   split_function=None
   if not in_set[column].dtype.name == "category":
      # assume it to be numerical if not category
      split_function=lambda in_set:in_set[column]>=value
   else:
      split_function=lambda in_set:in_set[column]==value
                                   
   # Divide the rows into two sets and return them
   set1= in_set[split_function(in_set)].copy()
   set2= in_set[np.invert(split_function(in_set))].copy()
   return (set1,set2)

##    Load Data     

In [62]:
dat_car = pa.read_csv('car.data.csv', sep=",")
dat_car.dtypes
for i in dat_car.columns.values:
    dat_car[i] = dat_car[i].astype('category')
    
dat = dat_car
target_col = "rating"

# shuffle data
np.random.seed(42)
dat = dat.reindex(np.random.permutation(dat.index))

# split data into training and test set -- this is absolutely central to fitting a ML model
# if you are not shure why, ask your tutor (its going to be on the final exam)
split = int(dat.shape[0]/100*20)

dat_test = dat.iloc[0:split]
dat_train = dat.iloc[(split+1):dat.shape[0]]

Tree_node.original_Data = dat_train

# Task 0
Look at the dataset. The dataset contains information used cars that are for sale including a rating of the offer.

In [63]:
# print('shape')
# print(dat_train.shape)

# print('columns')
# print(list(dat_train.columns))

# print('labels: ')
# print(list(dat_train.rating.unique()))


# In this exercise we will fit a model to the remainig six variables in order to predict "rating", our goal variable.
# You will have to add your own code at each failing assert like this one
# assert False, "Please comment me"

## Task 1
Fit a tree stump: in this task you will fit a tree stump (i.e. a Decision Tree of depth 1) to the used cars data. You will have to find the partition of the input dataset that yields the biggest gini score gain with respect to the goal variable. you will accomplish this by exhaustive search i.e. try every possible partition.

# Input Parameters:
1. in_set: Training data (features)
2. traget_col: Ground truth labels
3. weights: Weight of the samples 

# Outputs:
1. best_split: A tuple including the most discriminant (best) feature for splitting, splitting value and labels
2. best_sets: A tuple of two sets, which are the outputs of "divideset" function. These sets are the result of dividing the training data based on the best split


In [64]:
def find_best_split(in_set, target_col, weights=None):
    # in_set = datas
    # target_col = ratings

    # compute the giniy score for the unpartitioned dataset
    # in_score = 0.6839147891222874
    in_score = gini_impurity(in_set, target_col, weights)
    
    best_gain = 0
    best_split = None
    best_sets = None
    
    # try every column
    for act_col in in_set.columns.values:
        # ignore goal variable - otherwise its trivial
        if act_col == target_col: continue

        # construct a list of unique values for this variable 
        column_values = set(in_set[act_col])

        # Try every possible split of the dataset w.r.t. the current variable
        # save the split that yielded the hightest gini gain
        # The gini-gain of a partition is defined as follows (assume set is partitioned into part_1 and part_2)
        # gain = gini_impurity(set)-p_1*gini_ipurity(part_1)-(1-p_1)*gini_inpurity(part_2)
        # where p_1 is nrows(part_1)/nrows(set)
        # Hint: see Tree_node comments for the definition of the split
        
        for column_value in column_values:
            # z.Bsp. act_col = buy_price
            # z.Bsp. column_value = high
            # z.Bsp. sets = set1 sind alle zu column_value high, set2 ist der ganze rest
            
            sets = divideset(in_set, act_col, column_value)

            p_1 = len(sets[0].values)/len(in_set.values)
            gain = in_score - p_1 * gini_impurity(sets[0], target_col, weights) - (1 - p_1) * gini_impurity(sets[1], target_col, weights)
       
            if gain > best_gain:
                best_gain = gain
                best_split = (act_col, column_value, target_col)
                best_sets = sets

    # best_split = ('safety', 'low', 'rating')
    return best_split, best_sets


# Fitting a stump is trivial when you have found the best split
split, sets = find_best_split(dat_train, target_col)
stump = Tree_node(split,sets[0],sets[1])


probs.index
CategoricalIndex(['unacc', 'acc', 'good', 'vgood'], categories=['acc', 'good', 'unacc', 'vgood'], ordered=False, dtype='category')
la1
<function gini_impurity.<locals>.<lambda> at 0x106b11268>
prob_obs
[0.67027027 0.67027027 0.67027027 ... 0.24555985 0.67027027 0.24555985]
prob_obs
[0.44926224 0.44926224 0.44926224 ... 0.06029964 0.44926224 0.06029964]
weights
[0.0007722 0.0007722 0.0007722 ... 0.0007722 0.0007722 0.0007722]
1-sum(weights*prob_obs)
0.6839147891222874
probs.index
CategoricalIndex(['unacc', 'acc'], categories=['acc', 'good', 'unacc', 'vgood'], ordered=False, dtype='category')
la1
<function gini_impurity.<locals>.<lambda> at 0x1a1481c488>
prob_obs
[0.75875486 0.75875486 0.75875486 0.75875486 0.24124514 0.24124514
 0.75875486 0.75875486 0.75875486 0.24124514 0.24124514 0.24124514
 0.75875486 0.24124514 0.24124514 0.75875486 0.75875486 0.75875486
 0.75875486 0.75875486 0.75875486 0.75875486 0.75875486 0.24124514
 0.24124514 0.75875486 0.75875486 0.75875486 0.241

probs.index
CategoricalIndex(['unacc', 'acc', 'good', 'vgood'], categories=['acc', 'good', 'unacc', 'vgood'], ordered=False, dtype='category')
la1
<function gini_impurity.<locals>.<lambda> at 0x1a147f08c8>
prob_obs
[0.64836224 0.64836224 0.64836224 ... 0.24662813 0.64836224 0.24662813]
prob_obs
[0.42037359 0.42037359 0.42037359 ... 0.06082544 0.42037359 0.06082544]
weights
[0.00096339 0.00096339 0.00096339 ... 0.00096339 0.00096339 0.00096339]
1-sum(weights*prob_obs)
0.7121513100164509
probs.index
CategoricalIndex(['unacc', 'acc'], categories=['acc', 'good', 'unacc', 'vgood'], ordered=False, dtype='category')
la1
<function gini_impurity.<locals>.<lambda> at 0x106b3b620>
prob_obs
[0.75301205 0.75301205 0.75301205 0.75301205 0.75301205 0.24698795
 0.75301205 0.24698795 0.24698795 0.75301205 0.75301205 0.75301205
 0.75301205 0.75301205 0.24698795 0.75301205 0.75301205 0.75301205
 0.75301205 0.75301205 0.75301205 0.24698795 0.75301205 0.75301205
 0.75301205 0.24698795 0.75301205 0.75301205

prob_obs
[0.64174455 0.64174455 0.64174455 0.64174455 0.2450675  0.64174455
 0.64174455 0.2450675  0.64174455 0.0529595  0.64174455 0.64174455
 0.2450675  0.0529595  0.64174455 0.2450675  0.64174455 0.64174455
 0.64174455 0.64174455 0.64174455 0.64174455 0.06022845 0.64174455
 0.64174455 0.64174455 0.06022845 0.64174455 0.64174455 0.64174455
 0.64174455 0.64174455 0.64174455 0.64174455 0.64174455 0.64174455
 0.2450675  0.2450675  0.2450675  0.64174455 0.2450675  0.64174455
 0.2450675  0.64174455 0.64174455 0.64174455 0.64174455 0.64174455
 0.64174455 0.2450675  0.64174455 0.0529595  0.64174455 0.06022845
 0.2450675  0.64174455 0.64174455 0.2450675  0.06022845 0.64174455
 0.2450675  0.0529595  0.06022845 0.06022845 0.06022845 0.2450675
 0.64174455 0.64174455 0.2450675  0.2450675  0.06022845 0.64174455
 0.2450675  0.06022845 0.64174455 0.64174455 0.64174455 0.2450675
 0.2450675  0.2450675  0.64174455 0.64174455 0.2450675  0.64174455
 0.64174455 0.64174455 0.2450675  0.64174455 0.64174455

[0.70425532 0.70425532 0.70425532 0.70425532 0.70425532 0.70425532
 0.70425532 0.70425532 0.70425532 0.25425532 0.70425532 0.70425532
 0.25425532 0.0212766  0.70425532 0.70425532 0.70425532 0.70425532
 0.70425532 0.02021277 0.70425532 0.25425532 0.25425532 0.70425532
 0.70425532 0.70425532 0.70425532 0.70425532 0.70425532 0.25425532
 0.25425532 0.25425532 0.70425532 0.25425532 0.70425532 0.70425532
 0.70425532 0.70425532 0.70425532 0.70425532 0.70425532 0.70425532
 0.70425532 0.70425532 0.25425532 0.70425532 0.02021277 0.25425532
 0.70425532 0.70425532 0.25425532 0.02021277 0.70425532 0.70425532
 0.70425532 0.70425532 0.25425532 0.70425532 0.70425532 0.70425532
 0.25425532 0.25425532 0.02021277 0.70425532 0.25425532 0.70425532
 0.25425532 0.02021277 0.70425532 0.70425532 0.70425532 0.70425532
 0.25425532 0.25425532 0.70425532 0.70425532 0.70425532 0.70425532
 0.70425532 0.70425532 0.70425532 0.70425532 0.70425532 0.70425532
 0.70425532 0.25425532 0.70425532 0.70425532 0.70425532 0.0212

prob_obs
[0.68961864 0.68961864 0.68961864 0.68961864 0.68961864 0.23622881
 0.68961864 0.68961864 0.03283898 0.68961864 0.23622881 0.23622881
 0.68961864 0.23622881 0.68961864 0.68961864 0.68961864 0.68961864
 0.68961864 0.68961864 0.23622881 0.23622881 0.68961864 0.68961864
 0.04131356 0.68961864 0.68961864 0.68961864 0.68961864 0.68961864
 0.68961864 0.23622881 0.23622881 0.68961864 0.23622881 0.68961864
 0.68961864 0.68961864 0.68961864 0.68961864 0.23622881 0.68961864
 0.68961864 0.03283898 0.04131356 0.68961864 0.23622881 0.68961864
 0.23622881 0.68961864 0.23622881 0.68961864 0.68961864 0.68961864
 0.03283898 0.68961864 0.04131356 0.04131356 0.68961864 0.68961864
 0.68961864 0.23622881 0.23622881 0.68961864 0.23622881 0.23622881
 0.68961864 0.68961864 0.68961864 0.23622881 0.68961864 0.68961864
 0.68961864 0.68961864 0.68961864 0.68961864 0.23622881 0.23622881
 0.23622881 0.68961864 0.68961864 0.23622881 0.68961864 0.68961864
 0.23622881 0.68961864 0.68961864 0.68961864 0.236228

...

 0.4        0.11724138 0.4       ]
prob_obs
[0.18086934 0.16       0.01374554 0.16       0.16       0.01374554
 0.16       0.18086934 0.00330295 0.16       0.16       0.18086934
 0.18086934 0.18086934 0.16       0.18086934 0.18086934 0.16
 0.18086934 0.01374554 0.00330295 0.18086934 0.16       0.00330295
 0.16       0.18086934 0.18086934 0.01374554 0.00330295 0.00330295
 0.16       0.16       0.16       0.18086934 0.18086934 0.18086934
 0.16       0.16       0.18086934 0.18086934 0.18086934 0.18086934
 0.16       0.01374554 0.18086934 0.16       0.16       0.18086934
 0.16       0.16       0.18086934 0.16       0.18086934 0.18086934
 0.18086934 0.16       0.16       0.01374554 0.16       0.18086934
 0.01374554 0.16       0.16       0.00330295 0.18086934 0.16
 0.01374554 0.18086934 0.00330295 0.16       0.18086934 0.18086934
 0.16       0.18086934 0.18086934 0.18086934 0.16       0.18086934
 0.01374554 0.01374554 0.18086934 0.18086934 0.16       0.18086934
 0.18086934 0.18086934 0.18086

[0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279
 0.00116279 0.00116279 0.00116279 0.00116279 0.00116279 0.0011

prob_obs
[0.5097589  0.5097589  0.5097589  0.5097589  0.5097589  0.36509759
 0.5097589  0.5097589  0.36509759 0.05855339 0.5097589  0.36509759
 0.36509759 0.05855339 0.5097589  0.36509759 0.5097589  0.5097589
 0.06659013 0.5097589  0.36509759 0.36509759 0.5097589  0.06659013
 0.5097589  0.5097589  0.5097589  0.5097589  0.5097589  0.5097589
 0.36509759 0.36509759 0.36509759 0.36509759 0.36509759 0.5097589
 0.5097589  0.5097589  0.5097589  0.5097589  0.36509759 0.5097589
 0.05855339 0.06659013 0.5097589  0.36509759 0.5097589  0.36509759
 0.5097589  0.06659013 0.36509759 0.5097589  0.5097589  0.36509759
 0.05855339 0.06659013 0.06659013 0.06659013 0.36509759 0.36509759
 0.36509759 0.36509759 0.06659013 0.36509759 0.36509759 0.06659013
 0.5097589  0.5097589  0.5097589  0.5097589  0.36509759 0.36509759
 0.36509759 0.5097589  0.5097589  0.36509759 0.5097589  0.5097589
 0.36509759 0.5097589  0.5097589  0.5097589  0.5097589  0.36509759
 0.05855339 0.5097589  0.5097589  0.5097589  0.36509759 0.

prob_obs
[0.5940367  0.5940367  0.5940367  0.5940367  0.33027523 0.5940367
 0.5940367  0.5940367  0.5940367  0.5940367  0.5940367  0.5940367
 0.07568807 0.5940367  0.5940367  0.5940367  0.33027523 0.33027523
 0.33027523 0.33027523 0.5940367  0.5940367  0.5940367  0.5940367
 0.33027523 0.5940367  0.33027523 0.07568807 0.33027523 0.33027523
 0.07568807 0.33027523 0.07568807 0.5940367  0.33027523 0.33027523
 0.5940367  0.5940367  0.5940367  0.33027523 0.5940367  0.5940367
 0.5940367  0.33027523 0.5940367  0.5940367  0.5940367  0.33027523
 0.33027523 0.33027523 0.5940367  0.5940367  0.5940367  0.5940367
 0.5940367  0.5940367  0.07568807 0.5940367  0.5940367  0.5940367
 0.5940367  0.33027523 0.5940367  0.5940367  0.5940367  0.33027523
 0.5940367  0.33027523 0.5940367  0.5940367  0.5940367  0.5940367
 0.33027523 0.5940367  0.5940367  0.5940367  0.33027523 0.33027523
 0.33027523 0.33027523 0.33027523 0.5940367  0.33027523 0.5940367
 0.5940367  0.5940367  0.33027523 0.5940367  0.5940367  0.594

[0.70896391 0.70896391 0.70896391 0.70896391 0.70896391 0.20256112
 0.70896391 0.05937136 0.70896391 0.20256112 0.20256112 0.05937136
 0.20256112 0.70896391 0.70896391 0.70896391 0.70896391 0.70896391
 0.70896391 0.02910361 0.20256112 0.20256112 0.70896391 0.70896391
 0.70896391 0.70896391 0.70896391 0.70896391 0.70896391 0.70896391
 0.70896391 0.70896391 0.20256112 0.70896391 0.70896391 0.70896391
 0.70896391 0.20256112 0.70896391 0.70896391 0.05937136 0.70896391
 0.02910361 0.70896391 0.20256112 0.70896391 0.02910361 0.20256112
 0.70896391 0.70896391 0.70896391 0.05937136 0.02910361 0.70896391
 0.02910361 0.70896391 0.70896391 0.70896391 0.20256112 0.70896391
 0.70896391 0.70896391 0.20256112 0.70896391 0.70896391 0.20256112
 0.70896391 0.70896391 0.70896391 0.70896391 0.70896391 0.70896391
 0.20256112 0.70896391 0.20256112 0.70896391 0.70896391 0.70896391
 0.70896391 0.70896391 0.70896391 0.70896391 0.70896391 0.20256112
 0.70896391 0.70896391 0.70896391 0.70896391 0.05937136 0.7089

## Task 2
Compute the confusion matrix and the correct classification percentage for your tree.

# Input Parameters:
1. in_data: Data samples (features)   
2. target_col: Data labels 
3. tree: A trained tree for evaluating the samples

# Outputs:
1. conf_mat: Confusion matrix of the decisions based on the input tree  
2. p_correct: Probability of correct decisions


In [49]:
def conf_matrix(in_data, target_col, tree):
  
    # levels available in the target variable (in this example 4)
    # levels = count( acc, good, unacc, vgood )
    levels = uniquecounts(in_data, target_col).keys()
    
    # confusion matrix itself
    conf_mat =  np.zeros((len(levels),len(levels)))
    
    # percentage of correct classifications
    # level_values = unacc, acc, good, vgood
    level_values = levels.categories.values.tolist()

    p_correct = 0
    
    for index, row in in_data.iterrows():
        # example of row:         
        # buy_price        med
        # maintenance      med
        # doors          5more
        # persons            4
        # lug_space        med
        # safety          high
        # rating         vgood
        
        # example of result: 
        # unacc
        
        result = tree.classify(row)
        
        conf_mat[level_values.index(result)][level_values.index(row.rating)] += 1
 
        # Correct data are in the diagonals
        correct = sum(conf_mat[i][i] for i in range(len(levels)))

        p_correct = correct / len(in_data)

    return conf_mat, p_correct

# Build confusion Matrix with training data
conf_mat_stump_train, p_correct_stump_train = conf_matrix(dat_train, target_col, stump)

# Build confusion Matrix with test data
conf_mat_stump_test, p_correct_stump_test = conf_matrix(dat_test, target_col, stump)

## Task 3
Recursively build a tree of variable depth.

# Input Parameters:
1. in_data: Training data (features)
2. traget_col: Ground truth labels
3. max_depth: Maximum length of the tree
3. weights: Weight of the samples

# Output:
An instance of the "Tree_node" class initialized by a split (output of the "find_best_split" function) in addition to the
right and left children (recursive call of the "train_tree" function for two subset of training data) 

In [52]:
def train_tree(in_data, target_col, max_depth=99, weigths = None):
    # To recursively build a decision tree you have to do two things:
    # - if you hit your stopping criterions just return in_data (there are two reasons that stop the recursion)
    # - othervise find the best split and call this methon on both set partitions
    
    # Stopping criterion
    if max_depth == 0 or in_data is None:
        return in_data
    
    # Call find_best_split
    split, sets = find_best_split(in_data, target_col, weigths)
    
    # Stopping criterion
    if sets is None:
        return in_data
    
    # Recursive calls
    right_child = train_tree(sets[0], target_col, max_depth-1, weigths)
    left_child = train_tree(sets[1], target_col, max_depth-1, weigths)

    return Tree_node(split, right_child, left_child)
    
# Tree of depth 5 (fully grown tree is pretty slow, but you can play around with the depth parameter and have a look
# at its influence on the classification performance)
depth5_tree = train_tree(dat_train, target_col, 5)

# Build confusion Matrix with training data
conf_mat_5_train, p_correct_5_train = conf_matrix(dat_train, target_col, depth5_tree)

# Build confusion Matrix with test data 
conf_mat_5_test, p_correct_5_test = conf_matrix(dat_test, target_col, depth5_tree)


train data
0.6702702702702703
test data
0.7191358024691358
train data
0.78996138996139
test data
0.8024691358024691
train data
0.78996138996139
test data
0.8024691358024691
train data
0.8208494208494208
test data
0.8333333333333334
train data
0.8409266409266409
test data
0.8672839506172839
train data
0.8841698841698842
test data
0.904320987654321
train data
0.905019305019305
test data
0.9197530864197531
train data
0.9397683397683397
test data
0.9290123456790124
train data
0.9675675675675676
test data
0.9691358024691358
train data
0.9806949806949807
test data
0.9629629629629629
train data
0.993050193050193
test data
0.9814814814814815
train data
0.9953667953667954
test data
0.9876543209876543
train data
0.9984555984555985
test data
0.9845679012345679
train data
1.0
test data
0.9845679012345679
train data
1.0
test data
0.9845679012345679
train data
1.0
test data
0.9845679012345679
Unexpected error: <class 'KeyboardInterrupt'>


KeyboardInterrupt: 

## Task 4
In this task you repeat task 3 and 2 but use the popular ML library scikit-learn instead of your own with a custom implementation.

In [36]:
# Encoding the variable for use with sklearn - the data has to be encoded this way for sklearn dont worry about the
# next 3 lines - just use the _encoded version when you pass data to sklearn
d = defaultdict(LabelEncoder)
dat_train_encoded = dat_train.apply(lambda x: d[x.name].fit_transform(x))
dat_test_encoded = dat_test.apply(lambda x: d[x.name].transform(x))

# Hints: have a Look at the DecisionTreeClassifier class. use criterion = "gini" and max_depth = 5 to make the results 
# comparable to task 3. You might have to take a look at the sklearn documentation.
# Attention: you pass data to sklearn you have to remove the target variable - otherwise sklearn will use it for 
# the prediction (e.g. use: dat_train_encoded[dat_train_encoded.columns.difference([target_col])] as training data)

tree = DecisionTreeClassifier(criterion='gini', max_depth=5)
cols_without_target = dat_train_encoded.columns.difference([target_col]);

tree.fit(dat_train_encoded[cols_without_target], dat_train_encoded[target_col])
predictions = tree.predict(dat_test_encoded[cols_without_target])

# inverse the encoding on the predictions and compute the confusion rate --> how does this compare to your own implementation?
predictions = d[target_col].inverse_transform(predictions)
sum(dat_test[target_col] == predictions)/float(len(predictions))


0.7191358024691358
;
0.7654320987654321
;
0.7777777777777778
;
0.8333333333333334
;
0.8148148148148148
;
0.8425925925925926
;
0.8549382716049383
;
0.9567901234567902
;
0.9506172839506173
;
0.9444444444444444
;
0.9783950617283951
;
0.9382716049382716
;
0.9753086419753086
;
0.9506172839506173
;
0.9629629629629629
;
0.9783950617283951
;
0.9814814814814815
;
0.9691358024691358
;
0.9783950617283951
;
0.9814814814814815
;


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

# Task 5
In this task you will implement the adaboost algorithm to fit a number of trees and use their collective power
to build a better classifier. While you can use your own tree implementation i advise you to use DecisionTreeClassifier
for performance reasons (on run of adaboost will fit 50 trees). Using the following function you will fits decision trees to the data using adaboost:

# Input Parameters (ada_boost_trees):
1. in_data: Training data (features)
2. column: Ground truth labels 
3. depth: Depth of the individual trees
4. m: Number of trees (hypotheses) to fit

# Outputs (ada_boost_trees):
1. trees: A list of the fitted trees
2. importance = A list of the respective importances (weights) for the fitted trees. These values are used for final weighted decisions 

# Input Parameters (predict_boosted_trees):
1. trees: A list of the fitted trees
2. importance = A list of the respective importances (weights) for the fitted trees. These values are used for final weighted decisions 
3. obs: Data samples for evaluation

# Output (predict_boosted_trees):
1. trees: A list of the fitted trees
2. importance = A list of the respective importances (weights) for the fitted trees. These values are used for final weighted decisions. 

In [25]:
def ada_boost_trees(in_data, column, depth, m):
    trees = []
    importance = []
    
    N, _ = in_data.shape
    # initialize weights uniform
    w = np.ones(in_data.shape[0]) * float(1)/in_data.shape[0]
    
    for k in range(m):
        # fit tree using actual weights
        d_tree = DecisionTreeClassifier(criterion = "gini", max_depth=depth)
        d_tree = d_tree.fit(in_data[in_data.columns.difference([target_col])],  in_data[target_col], sample_weight=w)
        predictions = d_tree.predict(in_data[in_data.columns.difference([target_col])])            

        # compute the weighted error
        # i.e. sum up w but leave out each value, for which the prediciton is correct
        weighted_err = 0
        for i, v in enumerate(in_data[target_col].values):
            if v != predictions[i]:
                weighted_err += w[i]
                
        weighted_err /= sum(w)
        
        # if the weighted error is low enough we just stop
        if weighted_err < 1e-200:
            break
        
        
        # update weights
        # for each observation where the prediction is Correct update the corresponding weights
        # w[i] = w_[i] * weighted_err/(1-weighted_err)
        # update weights
        for i, v in enumerate(in_data[target_col].values):
            if v == predictions[i]:
                w[i] = w[i] * weighted_err/(1-weighted_err)
        
        
        #model importance
        model_imp = np.log(1-weighted_err)/(weighted_err)
        
        trees.append(d_tree)
        importance.append(model_imp)
        
    return trees, importance

# predicts the class of a number of observations, based on trees and importances returned by the above method
def predict_boosted_trees(trees, importance, obs):
    N, _ = obs.shape
    
    predictions_dir = dict()
    
    for (tree, model_imp) in zip(trees, importance):
        if model_imp == 0: continue

        predictions = tree.predict(obs)
        levels = set(predictions)

        for level in levels:
            if level in predictions_dir.keys():
                predictions_dir[level] += (predictions == level)*(model_imp)
            else:
                predictions_dir[level] = (predictions == level)*(model_imp)
    
    pred = np.zeros((N,len(predictions_dir.keys())))
    
    for k in predictions_dir.keys():
        pred[:,k]=predictions_dir[k]
    
    return np.argmin(pred, axis=1)

# based on the code in predict_boosted_trees, how does the prediction based on a number of boosted trees work?


# train boosted trees
trees, importance = ada_boost_trees(dat_train_encoded, target_col, 5, 50)

# predict using boosted trees
predictions = predict_boosted_trees(trees, importance,dat_test_encoded[dat_test_encoded.columns.difference([target_col])])
predictions = d[target_col].inverse_transform(predictions)

# compute prediction accuracy of the boosted trees
sum(dat_test[target_col] == predictions)/float(len(predictions))


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9753086419753086
0.9753086419753086


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9753086419753086
0.9753086419753086


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9753086419753086
0.9753086419753086


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9753086419753086
0.9753086419753086
0.9753086419753086
0.9753086419753086


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Task 6
Compare our Adaboosted Trees versus sklearn. Fit an sklearn AdaBoostClassifier using a DecisionTreeClassifier as base classifier.

In [40]:
seed = 42


for i in range(1, 10):
    ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='gini', max_depth=i, splitter='random'), n_estimators=50, random_state=seed)
    ada.fit(dat_train_encoded[dat_train_encoded.columns.difference([target_col])],  dat_train_encoded[target_col])
    predictions = ada.predict(dat_test_encoded[dat_test_encoded.columns.difference([target_col])])   

    predictions = d[target_col].inverse_transform(predictions)
    print(sum(dat_test[target_col] == predictions)/float(len(predictions)))
    print(';')

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7808641975308642
;
0.9012345679012346
;
0.9228395061728395
;


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9475308641975309
;
0.9783950617283951
;
0.9660493827160493
;
0.9876543209876543
;
0.9814814814814815
;
0.9845679012345679
;


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
